# Named Entity Recognition - Rozpoznawanie encji nazwanych / tagowanie sekwencji

Dotychczas na większości zajęć rozważaliśmy problem klasyfikacji, w którym całym dokumentom przypisywalśmy pojedynczą etykietę (sentyment związany z dokumentem, informacja o tym, czy tekst jest spamowy, etykieta mówiąca o tym w jakim języku napisany jest dokument). Warto jednak również wspomnieć o tzw. tagowaniu sekwencji, które dla każdego elementu sekwencji (słowa) nadaje odpowiednią etykietę.

Gdzie taka procedura ma zastosowanie? Wymieńmy kilka przykładów 
<ol>
    <li>Wykrywanie wyrażeń dotyczących miejsc, ludzi, czasu, lokalizacji itp. - każde kolejne słowo tagowane jest informacją mówiącą o tym, czy dane słowo jest częścią pożądanego przez nas typu (np. częścią lokalizacji), czy nie (np. z użyciem kodowania IOB, o którym mówiliśmy przy okazji zajęć dotyczących CONLL)</li>
    <li>Tagowanie częściami mowy - każde słowo otrzymuje etykietę mówiącą o tym jaka część mowy reprezentowana jest przez aktualny token.</li>
    <li>Wykrywanie ważnych z naszego punktu widzenia fraz (nazwy produktów, technologii itp.)</li>
    <li>...</li>
</ol>

Mówiąc o encjach nazwanych (Named Entities) - mówimy o frazach, którym nadaliśmy określony typ, np: "01.06.2018" - typ "Data", "Poznań, Polska" - typ "Lokalizacja", "GeForce 1080 GTX Ultra" - typ "Sprzęt Komputerowy".

Dzisiejsze laboratoria dotyczyć będą właśnie tagowania sekwencji słów z użyciem tzw. NERa (Named Entity Recognizer - Detektor encji nazwanych).

In [1]:
!pip install python-crfsuite

In [2]:
import nltk
nltk.download('popular')
nltk.download('conll2002')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

## Gotowy NER - SpaCy

W przypadku, kiedy chcemy dla tekstów w języku angielskim w szybki sposób wyszukać bardzo popularne frazy typu: data lub lokalizacja - jest bardzo duża szansa, że możemy wykorzystać gotowe modele narzędzi takich jak NLTK, czy SpaCy.

**Zadanie1 (1 punkt)**: 
<ol>
<li>Korzystając z dokumentacji SpaCy dotyczącej NERa (https://spacy.io/usage/linguistic-features#section-named-entities) Wyświetl encje nazwane, znalezione w tekście ze zmiennej 'text'. Wykorzystaj model o nazwie 'en_core_web_md'.</li>
<li>Sprawdź, jakiego typu encje wyszukiwane są przez NER ze SpaCy w standardowych modelach dla języka angielskiego (https://spacy.io/api/annotation)</li>



In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')
text = "Jim Gates bought 300 shares of Acme Corp. in 2006."
for x in nlp(text).ents:
    print(x.text, x.label_) 

Jim Gates PERSON
300 CARDINAL
Acme Corp. ORG
2006 DATE


Oczekiwany rezultat:
<pre>
Jim Gates PERSON
300 CARDINAL
Acme Corp. ORG
2006 DATE
</pre>

## Gotowy NER - NLTK

Również NLTK udostępnia nam wykrywanie encji nazwanych. Spróbujmy to zrobić.

**Zadanie2 (1 punkt):**
Korzystając z dokumentacji NLTK wykonaj wykrywanie encji nazwanych z przykładowego zdania wykonując sekwencję kroków:
<ol>
    <li>Wykorzystaj funkcję word_tokenize do podziału zdania na tokeny.</li>
    <li>Na reprezentacji zwróconej z kroku: "podział na tokeny", wykonaj POS-tagging (nadaj każdemu tokenowi część mowy) z użyciem funkcji pos_tag()</li>
    <li>Na wyniku tagowania częściami mowy - wykonaj funkcję ne_chunk() do wykrycia encji.</li>
    <li>Jeśli wyświetlisz wynik funkcji ne_chunk zobaczysz coś co bardzo luźno przypomina drzewo, użyj funkcji tree2conlltags() aby zamienić tę reprezentację, na trójki CONLL i wyświetl wynik tej funkcji jako rozwiązanie zadania</li>
</ol>

Funkcje, których należy użyć to word_tokenize, pos_tag, ne_chunk, tree2conlltags. Wszystkie zostały już zaimportowane.

In [4]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import tree2conlltags

text = "Jim Gates bought 300 shares of Acme Corp. in 2006."
tree2conlltags(ne_chunk(pos_tag(word_tokenize(text))))

[('Jim', 'NNP', 'B-PERSON'),
 ('Gates', 'NNP', 'B-PERSON'),
 ('bought', 'VBD', 'O'),
 ('300', 'CD', 'O'),
 ('shares', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('Acme', 'NNP', 'B-ORGANIZATION'),
 ('Corp.', 'NNP', 'I-ORGANIZATION'),
 ('in', 'IN', 'O'),
 ('2006', 'CD', 'O'),
 ('.', '.', 'O')]

Oczekiwany rezultat:

<pre>
[('Jim', 'NNP', 'B-PERSON'), ('Gates', 'NNP', 'B-PERSON'), ('bought', 'VBD', 'O'), ('300', 'CD', 'O'), ('shares', 'NNS', 'O'), ('of', 'IN', 'O'), ('Acme', 'NNP', 'B-ORGANIZATION'), ('Corp.', 'NNP', 'I-ORGANIZATION'), ('in', 'IN', 'O'), ('2006', 'CD', 'O'), ('.', '.', 'O')]
</pre>
## Własny NER - trening z użyciem algorytmu CRF (Conditional Random Fields)

Wykrywacze encji wytrenowane są do odnajdywania popularnych typów fraz (Daty, Lokalizacje, Osoby, ...). Co jednak, kiedy chcielibyśmy wykrywać zdefiniowane przez nas typy danych (np. sprzęt komputerowy), które nie są domyśnie wspierane przez istniejące modele? Musielibyśmy wytrenować własnego NERa. Użyjmy paczki 'pycrfsuite' do tego celu.

PyCRFSuite implementuje algorytm CRF - bardzo wydajny algorytm, który potrafi uczyć się tagowania poszczególnych słów z użyciem np. kodowania IOB. Aby rozróżnić różne rodzaje encji, często tagi "I" i "B" kodowania IOB opatruje się dodatkowym sufiksem. Np. B-Date - oznacza początek daty, a I-Location - kontynuację frazy zawierającej lokację.

Ponieważ to czy dane słowo jest encją nazwaną zależy zarówno od tego jak dane słowo wygląda, jak i od słów poprzedzających i następujących po aktualnym - w opisie cech CRFów również uwzględnia się informacje o okalających słowach.

**Zadanie (2 punkty)** Wytrenuj model, który będzie tagował poszczególne słowa w tekście z użyciem pycfrsuite. Aby to zrobić, wykonaj podzadania w krokach poniżej.

Nasz NER będzie się uczyć etykiet na zbiorze tekstów hiszpańskich, które poddane są podziałowi na zdania, tokenizacji, tagowaniem częściami mowy i etykietami encji do wykrycia w formacie IOB. 

In [5]:
import nltk
import sklearn
import pycrfsuite

train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train')) # załaduj korpus treningowy dla języka hiszpańskiego
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))  # załaduj korpus testowy dla języka hiszpańskiego
train_sents[2] # wyświetla przykładowe zdanie, aby zobaczyć jak reprezentowane są dane

[('El', 'DA', 'O'),
 ('Abogado', 'NC', 'B-PER'),
 ('General', 'AQ', 'I-PER'),
 ('del', 'SP', 'I-PER'),
 ('Estado', 'NC', 'I-PER'),
 (',', 'Fc', 'O'),
 ('Daryl', 'VMI', 'B-PER'),
 ('Williams', 'NC', 'I-PER'),
 (',', 'Fc', 'O'),
 ('subrayó', 'VMI', 'O'),
 ('hoy', 'RG', 'O'),
 ('la', 'DA', 'O'),
 ('necesidad', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('tomar', 'VMN', 'O'),
 ('medidas', 'NC', 'O'),
 ('para', 'SP', 'O'),
 ('proteger', 'VMN', 'O'),
 ('al', 'SP', 'O'),
 ('sistema', 'NC', 'O'),
 ('judicial', 'AQ', 'O'),
 ('australiano', 'AQ', 'O'),
 ('frente', 'RG', 'O'),
 ('a', 'SP', 'O'),
 ('una', 'DI', 'O'),
 ('página', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('internet', 'NC', 'O'),
 ('que', 'PR', 'O'),
 ('imposibilita', 'VMI', 'O'),
 ('el', 'DA', 'O'),
 ('cumplimiento', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('los', 'DA', 'O'),
 ('principios', 'NC', 'O'),
 ('básicos', 'AQ', 'O'),
 ('de', 'SP', 'O'),
 ('la', 'DA', 'O'),
 ('Ley', 'NC', 'B-MISC'),
 ('.', 'Fp', 'O')]

**Zadanie 2a (1 punkt)** Tworzenie cech. PyCRFSuite oczekuje, że każde słowo opisane będzie zestawem odpowiednich cech w formie pythonowego słownika. Uzupełnij kod funkcji word2features (sekcje TODO) tak, aby stworzyć odpowiednie cechy zgodnie z nazwami i komentarzami do poszczególnych pól.

In [0]:
def word2features(sent, i):
    word = sent[i][0]  # sent[i] ma postać np. ('Ley', 'NC', 'B-MISC'); Indeks 0 oznacza pierwszy element z nawiasów (tupli), czyli w tym przypadku 'Ley'
    postag = sent[i][1] # sent[i] ma postać np. ('Ley', 'NC', 'B-MISC'); Indeks 0 oznacza pierwszy element z nawiasów (tupli), czyli w tym przypadku 'NC'
    
    features = {      # cechy aktualnego słowo
        'bias': 1.0,
        'lowercase_word': word.lower(), # TODO, tutaj słowo małymi literami
        'word_last_3_chars': word[-3:], # TODO, tutaj ostatnie 3 znaki słowa
        'word_last_2_chars': word[-2:], # TODO, tutaj ostatnie 2 znaki słowa
        'word_is_uppercase': word.isupper(), # TODO, tutaj flaga (True/False), czy słowo jest uppercase
        'word_is_digit': word.isdigit(), # TODO, tutaj flaga (True/False), czy słowo jest liczbą
        'postag': postag, # TODO, tutaj pos-tag (patrz początek definicji funkcji)
        'postag_first_two_chars': postag[:2], # TODO, tutaj pierwsze 2 znaki pos-tagu  
    }
    if i > 0:         # jeśli nasze słowo nie jest pierwszym w zdaniu - dodajmy do zbioru naszych cech cechy poprzedniego tokenu
        word1 = sent[i-1][0]    # poprzednie słowo
        postag1 = sent[i-1][1]  # poprzedni pos-tag
        
        features.update({       # funkcja update() na słowniku dopisuje dodatkowe atrybuty do istniejącego słownika
            'previous_word_lower': word1.lower(), # TODO, tutaj poprzednie słowo małymi literami
            'previous_word_is_upppercase': word1.isupper(), # TODO, tutaj flaga (True/False), czy słowo jest uppercase
            'previous_word_postag': postag1, # TODO, tutaj pos-tag poprzedniego słowa 
            'previous_word_postag_first_two_chars': postag1[:2], # TODO, tutaj pierwsze 2 znaki pos-tagu  poprzedniego słowa
        })
    else:
        features['BOS'] = True   # jeśli to pierwszy token - ustawmy cechę BOS (Begin of Sentence) na True
        
    if i < len(sent)-1:          # Jeśli nasze słowo nie jest ostatnim - dodajmy do zbioru cech cechy następnego słowa 
        word1 = sent[i+1][0]     # następne słowo
        postag1 = sent[i+1][1]   # następny postag
        
        features.update({        # funkcja update() na słowniku dopisuje dodatkowe atrybuty do istniejącego słownika
            'next_word_is_lower': word1.islower(), # TODO, tutaj flaga - czy następne słowo małymi literami
            'next_word_is_upppercase': word1.isupper(), # TODO, tutaj flaga (True/False), czy słowo jest uppercase
            'next_word_postag': postag1, # TODO, tutaj pos-tag następnego słowa 
            'next_word_postag_first_two_chars': postag1[:2] # TODO, tutaj pierwsze 2 znaki pos-tagu  następnego słowa
        })
    else:
        features['EOS'] = True   # jeśli to ostatni token - ustawmy cechę EOS (End of Sentence) na True
                
    return features



def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))] # zamień każde słowo ze zdania na słownik cech

In [7]:
sent2features(train_sents[0])[0]

{'BOS': True,
 'bias': 1.0,
 'lowercase_word': 'melbourne',
 'next_word_is_lower': False,
 'next_word_is_upppercase': False,
 'next_word_postag': 'Fpa',
 'next_word_postag_first_two_chars': 'Fp',
 'postag': 'NP',
 'postag_first_two_chars': 'NP',
 'word_is_digit': False,
 'word_is_uppercase': False,
 'word_last_2_chars': 'ne',
 'word_last_3_chars': 'rne'}

Oczekiwany rezultat: 
<pre>
{'BOS': True,
 'bias': 1.0,
 'lowercase_word': 'melbourne',
 'next_word_lower': 'False',
 'next_word_is_upppercase': False,
 'next_word_postag': 'Fpa',
 'next_word_postag_first_two_chars': 'Fp',
 'postag': 'NP',
 'postag_first_two_chars': 'NP',
 'word_is_digit': False,
 'word_is_uppercase': False,
 'word_last_2_chars': 'ne',
 'word_last_3_chars': 'rne'}
</pre>
 
 **Zadanie 2b (1 punkt) - napisz ciała funkcji pomocniczych, które dla aktualnego zdania z train_sents i test_sents zwrócą:**
 <ul>
     <li>sent2labels - zwróci ciąg oczkiwanych etykiet dla każdego wyrazu. parametr sent jest listą słów, z których każde słowo opisane jest trójką: tekst słowa, pos-tag słowa, etykieta słowa; np. ('Abogado', 'NC', 'B-PER') </li>
     <li>sent2tokens - analogicznie do powyższego, jednak zamiast etykiet zwróci ciąg słów bez pos-tagów i etykiet.</li>
     <li>get_all_labels - funkcja, która ze zbioru wszystkich zdań treningowych wyświetli zbiór etykiet (zbiór, czyli bez powtórzeń). Funkcja pokaże nam ilu etykiet chcemy się nauczyć, aby móc ocenić trudność naszego problemu.</li>
 </ul>

In [8]:
def sent2labels(sent):
  return [x[2] for x in sent]

def sent2tokens(sent):
  return [x[0] for x in sent]

def get_all_labels(sents):
  # flatmap :)
  return {x[2] for xs in sents for x in xs}

print(sent2labels(train_sents[0]))
print(sent2tokens(train_sents[0]))
print(get_all_labels(train_sents))

['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']
['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']
{'B-MISC', 'I-PER', 'B-ORG', 'O', 'I-LOC', 'B-LOC', 'B-PER', 'I-MISC', 'I-ORG'}


Oczekiwany rezultat:
<pre>
['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']
['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']
{'I-PER', 'I-MISC', 'B-LOC', 'I-LOC', 'B-PER', 'B-MISC', 'I-ORG', 'B-ORG', 'O'}
</pre>

Uruchom poniższy kod i sprawdź czego nauczył się nasz NER.

In [9]:
X_train = [sent2features(s) for s in train_sents] # Stwórz cechy zbioru treningowego
y_train = [sent2labels(s) for s in train_sents]   # Pobierz etykiety zbioru treningowego

X_test = [sent2features(s) for s in test_sents]   # Stwórz cechy zbioru testowego
y_test = [sent2labels(s) for s in test_sents]     # Pobierz etykiety zbioru testowego

trainer = pycrfsuite.Trainer(verbose=False)    # stwórz obiekt trenujący

for xseq, yseq in zip(X_train, y_train):       # iteruj po zdaniach i etykietach
    trainer.append(xseq, yseq)                 # dopisuj do obiektu trenującego nasze dane
    
trainer.set_params({
    'c1': 1.0,   # parametr regularyzacyjny L1
    'c2': 1e-3,  # parametr regularyzacyjny L2
    'max_iterations': 50,  # maksymalna liczba iteracji
    # dodaj tranzycje, które nie są obserwowane ale są możliwe
    'feature.possible_transitions': True
})

trainer.train('conll2002-esp.crfsuite')       # wytrenuj model i zapisz do pliku!

tagger = pycrfsuite.Tagger()                  # stwórz tagger, który będzie nadawał etykiety naszej sekwencji
tagger.open('conll2002-esp.crfsuite')         # załaduj do niego wytrenowany model
example_sent = test_sents[0]                  # weź pierwsze z brzegu zdanie, które nie brało udziału w treningu
print(' '.join(sent2tokens(example_sent)), end='\n\n')   # wyświetl je...

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))  # zobacz, co generuje nasz model
print("Correct:  ", ' '.join(sent2labels(example_sent)))                # i to, czego oczekiwano!

La Coruña , 23 may ( EFECOM ) .

Predicted: B-LOC I-LOC O O O O B-ORG O O
Correct:   B-LOC I-LOC O O O O B-ORG O O


## Ekstrakcja fraz rzeczownikowych

Czasami chcielibyśmy z danego tekstu wyekstrahować nie tylko encje, na których nasz NER jest wytrenowany (pewien podzbiór kategorii), ale wszystkie frazy opisujące obiekty. Po co? przydać się to może np. przy tworzeniu tzw. chmury słów kluczowych, której przykład znajdziecie poniżej, bądź w problemach automatycznego odpowiadania na pytania.

<img src="./cloud.jpg"/>

Wydawałoby się, że aby tego dokonać, sensownym podejściem byłoby zidentyfikowanie wszystkich rzeczowników, np. w zdaniu "Ala ma piękny mały dom", rzeczownikami są: "Ala", "dom". 
Ograniczając się do rzeczowników, tracimy jednak ważne informacje, które opisują rzeczowniki, np. bardzo istotne może być zapamiętanie, że dom Ali jest piękny i mały.

Czy istnieje sposób automatycznego ekstrahowania całych tzw. fraz rzeczownikowych? (https://pl.wikipedia.org/wiki/Fraza_nominalna)

Oczywiście, z pomocą przychodzą narzędzia takie jak SpaCy czy NLTK.

**Zadanie4 (1 punkt)** Korzystając z dokumentacji SpaCy, zidentyfikuj wszystkie frazy rzeczownikowe (noun chunks) z zadanego zdania.

In [10]:
text = "Jim Gates bought 300 shares of Acme Corp and a tiny beautiful house."
for x in nlp(text).noun_chunks:
  print(x)

Jim Gates
300 shares
Acme Corp
a tiny beautiful house


Oczekiwany rezultat:

<pre>
Jim Gates
300 shares
Acme Corp
a tiny beautiful house
</pre>